In [2]:
import pandas as pd
from pathlib import Path
from basketball_reference_scraper.seasons import get_schedule, get_standings
from basketball_reference_scraper.box_scores import get_box_scores
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


In [3]:
data_path = Path('Data')
assert data_path.exists()

In [4]:
with open(data_path/'team_params.txt') as f:
    teams = f.readlines()
team_dict={}
for team in teams:
    splits = team.split(":")
    team_dict[splits[0].strip().title()] = splits[1].strip()
team_dict

{'Atlanta Hawks': 'ATL',
 'St. Louis Hawks': 'SLH',
 'Milwaukee Hawks': 'MIL',
 'Tri-Cities Blackhawks': 'TCB',
 'Boston Celtics': 'BOS',
 'Brooklyn Nets': 'BRK',
 'New Jersey Nets': 'NJN',
 'Chicago Bulls': 'CHI',
 'Charlotte Hornets (1988-2004)': 'CHH',
 'Charlotte Hornets (2014-Present)': 'CHO',
 'Charlotte Bobcats': 'CHA',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Fort Wayne Pistons': 'FWP',
 'Golden State Warriors': 'GSW',
 'San Francisco Warriors': 'SFW',
 'Philadelphia Warriors': 'PHI',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'Los Angeles Clippers': 'LAC',
 'San Diego Clippers': 'SDC',
 'Buffalo Braves': 'BUF',
 'Los Angeles Lakers': 'LAL',
 'Minneapolis Lakers': 'MIN',
 'Memphis Grizzlies': 'MEM',
 'Vancouver Grizzlies': 'VAN',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NOP',
 'New Orleans/Oklahoma City Hornets': 'NOK',
 'New Or

# Get Schedule

In [5]:
all_schedules = []
for year in range(2014,2022):
    print(year)
    year_schedule = get_schedule(year, playoffs=False)
    all_schedules.append(year_schedule)

2014
2015
2016
2017
2018
2019
2020
2021


In [6]:
schedule_20142021 = pd.concat(all_schedules)
len(schedule_20142021)

8980

In [7]:
playedgames_20142021 = schedule_20142021.dropna(how='any')
playedgames_20142021 = playedgames_20142021.replace({"Philadelphia 76ers":"Philadelphia 76Ers"})
playedgames_20142021 = playedgames_20142021.replace(team_dict)
playedgames_20142021 = playedgames_20142021.replace({'Charlotte Hornets':'CHO'})
len(playedgames_20142021)

8744

In [8]:
playedgames_20142021.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
0,2013-10-29,ORL,87,IND,97
1,2013-10-29,CHI,95,MIA,107
2,2013-10-29,LAC,103,LAL,116
3,2013-10-30,BRK,94,CLE,98
4,2013-10-30,BOS,87,TOR,93


In [9]:
# check if team names are replaced with abbreviation
for visitor in playedgames_20142021['VISITOR'].unique():
    assert len(visitor)==3, f"{visitor}'s team name needs fix"
for home in playedgames_20142021['HOME'].unique():
    assert len(visitor)==3, f"{home}'s team name needs fix"

# Build URLs

In [35]:
box_scores_urls=[]
gameids = []
for key,row in playedgames_20142021.iterrows():
    nums_to_join=[str(num) for num in [row['DATE'].year,f"{row['DATE'].month:02d}",f"{row['DATE'].day:02d}",0,row['HOME']]]
    url = "https://www.basketball-reference.com/boxscores/"+''.join(nums_to_join)+".html"
    gameids.append(''.join(nums_to_join))
    box_scores_urls.append(url)

In [36]:
box_scores_urls[:3]

['https://www.basketball-reference.com/boxscores/201310290IND.html',
 'https://www.basketball-reference.com/boxscores/201310290MIA.html',
 'https://www.basketball-reference.com/boxscores/201310290LAL.html']

In [12]:
len(box_scores_urls)

8744

In [37]:
playedgames_20142021['boxscores_url'] = box_scores_urls
playedgames_20142021['game_id'] = gameids
playedgames_20142021.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND
1,2013-10-29,CHI,95,MIA,107,https://www.basketball-reference.com/boxscores...,201310290MIA
2,2013-10-29,LAC,103,LAL,116,https://www.basketball-reference.com/boxscores...,201310290LAL
3,2013-10-30,BRK,94,CLE,98,https://www.basketball-reference.com/boxscores...,201310300CLE
4,2013-10-30,BOS,87,TOR,93,https://www.basketball-reference.com/boxscores...,201310300TOR


# Scrape four factors

In [40]:
tables = []
bad_games =[]
for key,row in tqdm(playedgames_20142021.iterrows()):
    try:
        response = requests.get(row['boxscores_url'])
        html = response.content
        html = html.decode()
        stat_html = html.replace('<!--', "")
        stat_html = stat_html.replace('-->', "")
        soup = BeautifulSoup(stat_html, 'html.parser')
        table = pd.read_html(str(soup.find_all('table',attrs={"id":"four_factors"})[0]))[0]
        table = table.droplevel(0,axis=1)
        table = table.rename({'Unnamed: 0_level_1':'Team'},axis=1)
        table['game_id'] = row['game_id']
        tables.append(table)
        if key%100==0: print(key)
    except:
        bad_games.append(row['game_id'])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300



In [41]:
temp_four_factors = pd.concat(tables)
temp_four_factors.head()

,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,game_id
0,ORL,95.0,0.435,14.9,27.7,0.065,91.6,201310290IND
1,IND,95.0,0.528,19.0,27.8,0.310,102.1,201310290IND
0,CHI,97.1,0.464,16.2,23.9,0.217,97.9,201310290MIA
1,MIA,97.1,0.590,17.5,14.3,0.306,110.2,201310290MIA
0,LAC,100.4,0.542,14.7,22.7,0.157,102.6,201310290LAL


In [42]:
temp_four_factors.to_pickle("Data/temp_four_factors.pkl")

In [43]:
temp_four_factors = pd.read_pickle("Data/temp_four_factors.pkl")

In [48]:
temp_four_factors

,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,game_id
0,ORL,95.0,0.435,14.9,27.7,0.065,91.6,201310290IND
1,IND,95.0,0.528,19.0,27.8,0.310,102.1,201310290IND
0,CHI,97.1,0.464,16.2,23.9,0.217,97.9,201310290MIA
1,MIA,97.1,0.590,17.5,14.3,0.306,110.2,201310290MIA
0,LAC,100.4,0.542,14.7,22.7,0.157,102.6,201310290LAL
...,...,...,...,...,...,...,...,...
1,WAS,96.6,0.651,12.6,16.7,0.157,125.3,202102020WAS
0,BOS,96.4,0.542,13.5,28.6,0.238,115.1,202102020GSW
1,GSW,96.4,0.548,10.6,13.3,0.193,111.0,202102020GSW
0,DET,99.6,0.468,12.8,25.0,0.194,105.4,202102020UTA


In [47]:
# handle the bad games
print(len(bad_games))
print(bad_games)

1
['201711110GSW']


# Join Four Factors with schedule to construct training data

In [47]:
# join on home team
home_joined=pd.merge(playedgames_20142021,table,left_on=['game_id','HOME'],right_on=['game_id','Team'])
home_joined

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND,IND,95.0,0.528,19.0,27.8,0.31,102.1


In [52]:
joined=pd.merge(home_joined,table,left_on=['game_id','HOME'],right_on=['game_id','Team'],suffixes=('_home','_away'))
joined.columns

Index(['DATE', 'VISITOR', 'VISITOR_PTS', 'HOME', 'HOME_PTS', 'boxscores_url',
       'game_id', 'Team_home', 'Pace_home', 'eFG%_home', 'TOV%_home',
       'ORB%_home', 'FT/FGA_home', 'ORtg_home', 'Team_away', 'Pace_away',
       'eFG%_away', 'TOV%_away', 'ORB%_away', 'FT/FGA_away', 'ORtg_away'],
      dtype='object')

In [53]:
joined

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id,Team_home,Pace_home,eFG%_home,...,ORB%_home,FT/FGA_home,ORtg_home,Team_away,Pace_away,eFG%_away,TOV%_away,ORB%_away,FT/FGA_away,ORtg_away
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND,IND,95.0,0.528,...,27.8,0.31,102.1,IND,95.0,0.528,19.0,27.8,0.31,102.1
